# Team ID : PNT2022TMID00692
Project Name : A Gesture-based Tool for Sterile Browsing ofRadiology Images

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='ZCrOtWul1v200DWCwCBXoW5dkX4fZ5rfrItF3fgF9sXW',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

bucket = 'agesturebasedtoolforsterilebrowsi-donotdelete-pr-5d9lix71rlbk2g'
object_key = 'Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

if not hasattr(streaming_body_1, "__iter__"): streaming_body_1.__iter__ = types.MethodType( __iter__, streaming_body_1 )

In [ ]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
import os
filenames = os.listdir(r'/home/wsuser/work/Dataset/train')

In [ ]:
x_train=train_datagen.flow_from_directory(r'/home/wsuser/work/Dataset/train',
                                          target_size=(64,64),
                                          batch_size=5,
                                          color_mode='grayscale',
                                          class_mode='categorical')

Found 594 images belonging to 6 classes.


In [ ]:
x_test = test_datagen.flow_from_directory(r'/home/wsuser/work/Dataset/test',
                                          target_size=(64, 64),
                                          batch_size=5,
                                          color_mode='grayscale',
                                          class_mode='categorical') 

Found 30 images belonging to 6 classes.


In [ ]:
import numpy as np

import tensorflow

from tensorflow.keras.models import Sequential

from tensorflow.keras import layers

from tensorflow.keras.layers import Dense,Flatten

from tensorflow.keras.layers import Conv2D,MaxPooling2D

from keras.preprocessing.image import ImageDataGenerator



In [ ]:
# Initializing the CNN
model = Sequential()

In [ ]:
# First convolution layer and pooling
model.add(Convolution2D(32, (3, 3), input_shape=(64, 64, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
# Second convolution layer and pooling
model.add(Convolution2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
# Flattening the layers i.e. input layer
model.add(Flatten())

In [ ]:
# Adding a fully connected layer, i.e. Hidden Layer
model.add(Dense(units=512 , activation='relu'))

In [ ]:
# softmax for categorical analysis, Output Layer
model.add(Dense(units=6, activation='softmax')) 

In [ ]:
model.summary()#summary of our model

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 6272)              0         
                                                                 
 dense_6 (Dense)             (None, 512)              

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit_generator(
    generator=x_train,steps_per_epoch=len(x_train),
    epochs=20,validation_data=x_test,validation_steps=len(x_test))

/tmp/wsuser/ipykernel_164/1772156357.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/20
119/119 [==============================] - 9s 75ms/step - loss: 0.6393 - accuracy: 0.7441 - val_loss: 0.4844 - val_accuracy: 0.8333
Epoch 2/20
119/119 [==============================] - 9s 73ms/step - loss: 0.3879 - accuracy: 0.8519 - val_loss: 0.3123 - val_accuracy: 0.9000
Epoch 3/20
119/119 [==============================] - 9s 75ms/step - loss: 0.2884 - accuracy: 0.8889 - val_loss: 0.3932 - val_accuracy: 0.9000
Epoch 4/20
119/119 [==============================] - 9s 73ms/step - loss: 0.2082 - accuracy: 0.9226 - val_loss: 0.2338 - val_accuracy: 0.9333
Epoch 5/20
119/119 [==============================] - 9s 75ms/step - loss: 0.1427 - accuracy: 0.9444 - val_loss: 0.2445 - val_accuracy: 0.9333
Epoch 6/20
119/119 [==============================] - 9s 74ms/step - loss: 0.1160 - accuracy: 0.9579 - val_loss: 0.1804 - val_accuracy: 0.9667
Epoch 7/20
119/119 [==============================] - 9s 75ms/step - loss: 0.1008 - accuracy: 0.9579 - val_loss: 0.0446 - val_accuracy: 0.9667

In [ ]:
# Save the model
model.save('gesture.h5')

In [ ]:
!tar -zcvf Gesture-based-Radiology-Images.tgz gesture.h5

gesture.h5


In [ ]:
ls 

Dataset/                            gesture.h5     test/
Gesture-based-Radiology-Images.tgz  model-bw.json  train/


In [ ]:
model_json = model.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)

# IBM DEPLOYMENT

In [ ]:
!pip install ibm_watson_machine_learning

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url": "https://us-south.ml.cloud.ibm.com",
                     "apikey": "dEexWL0BVPLMtaNCzN8FhFhkforJvMQa7XmtZjD5PiB_"
                   }
client = APIClient(wml_credentials)

In [ ]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [1]:
space_uid = guid_from_space_name(client,'Models')
print("Space UID = " + space_uid)

Space UID = a160ab50-1ff0-4ac0-98b5-b79655b2a468


In [ ]:
client.set.default_space(space_uid)

'SUCCESS'

In [ ]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [ ]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

a160ab50-1ff0-4ac0-98b5-b79655b2a468


In [ ]:
model_json=model.to_json()
with open("model-bw.json","w")as json_file:
    json_file.write(model_json)

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.applications.inception_v3 import preprocess_input
model=load_model("gesture.h5")

In [ ]:
img=image.load_img(r"/home/wsuser/work/Dataset/test/1/1.jpg",grayscale=True,target_size=(64,64))

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [ ]:
x=image.img_to_array(img)

In [ ]:
import numpy as np

x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape



(1, 64, 64, 1)

In [ ]:
pred=np.argmax(model.predict(x))

pred



1

In [ ]:
model.predict(x)

array([[1.02547594e-04, 8.94386649e-01, 1.05454221e-01, 5.64631482e-05,
        7.49410448e-20, 1.98021851e-12]], dtype=float32)